In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import itertools
import io
import sklearn.metrics

print(tf.__version__)

2.18.0


In [2]:
# get data
data = pd.read_csv("data/titanic.csv", delimiter=";")
data = data.dropna()
data = data.reset_index(drop=True)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
3,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
4,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S


In [3]:
# Selecting features:
y = data["Survived"]
cols_a = ["Pclass","Age","SibSp","Fare"]
cols_b = ["Sex","Embarked"]
X_a = data[cols_a]
X_b = data[cols_b]

# Some treatment:
X_b.loc[:,"Sex"] = X_b.loc[:,"Sex"].apply(lambda x: 1 if x == "male" else 0)
X_b.loc[:,"Sex"] = X_b.loc[:,"Sex"].astype(int)
X_b.loc[:,"Embarked"] = X_b.loc[:,"Embarked"].apply(lambda x: 0 if x == "S" else (1 if x == 'C' else 2))
X_b.loc[:,"Embarked"] = X_b.loc[:,"Embarked"].astype(int)

# Trasnforming to tensors
X_a = tf.constant(X_a.values.astype(np.float32))
ohe_sex = tf.one_hot(indices=X_b["Sex"].values.astype(np.int64), depth=len(X_b["Sex"].unique())-1)
ohe_embarked = tf.one_hot(indices=X_b["Embarked"].values.astype(np.int64), depth=len(X_b["Embarked"].unique())-1)
X = tf.concat([X_a, ohe_sex, ohe_embarked],1)
y = tf.constant(y) 



In [4]:
# train test split
p = 0.8
v = 0.1
i = int(p*len(X))
i_v = int((1-v)*i)

# split data into train andtest sets
X_train = X[:i_v]
X_val = X[i_v:i]
X_test = X[i:]

y_train = y[:i_v]
y_val = y[i_v:i]
y_test = y[i:]


In [5]:
class_names = ["Survived","Died"]

In [10]:
# Clear any logs from previous runs
!rd -r "./logs/"

# set random seed
tf.random.set_seed(43)
tf.keras.utils.set_random_seed(41)

# 1. create a model

model = tf.keras.Sequential([
    tf.keras.layers.Dense(4, activation="relu"),
    tf.keras.layers.Dense(2, activation="sigmoid")
], name="classification_example")

# 2. compile model
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.1),
    metrics = ["accuracy"]
)

def plot_confusion_matrix(cm, class_names):
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion Matrix of the Results")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=90)
    plt.yticks(tick_marks, class_names)

    labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('Real Class')
    plt.xlabel('Predicted Class')
    return figure

def plot_to_image(fig):
 # Saving the plot to a PNG in memory.
 bfr = io.BytesIO()
 plt.savefig(bfr, format='png')
 plt.close(fig)
 bfr.seek(0)
 # Converting PNG buffer to TF image
 image = tf.image.decode_png(bfr.getvalue(), channels=4)
 # Adding the batch dimension
 image = tf.expand_dims(image, 0)
 return image

def log_confusion_matrix(epoch, logs):
    # Using the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(X_val)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculating the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(y_val, test_pred)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
     tf.summary.image("Confusion Matrix", cm_image, step=epoch)

# Defining the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')

# 3. fit the model
model.fit(
    X_train,
    y_train,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[tensorboard_callback, cm_callback])

Epoch 1/100


O sistema n�o pode encontrar o arquivo especificado.
A pasta n�o est� vazia.
c:\Users\Windows 11\anaconda3\Lib\site-packages\keras\src\backend\tensorflow\nn.py:708: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/stepp - accuracy: 0.4062 - loss: 20.410
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 0.4269 - loss: 11.9781 - val_accuracy: 0.6667 - val_loss: 0.7485
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5938 - loss: 1.234
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.6359 - loss: 0.9354 - val_accuracy: 0.6667 - val_loss: 0.6403
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5938 - loss: 0.708
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.6359 - loss: 0.6685 - val_accuracy: 0.6667 - val_loss: 0.6369
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5938 - loss: 0.691
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.6359 - loss: 0.6605 - val_accuracy: 0.6667 - val_loss: 0.6387
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5938 - loss: 0.679
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.6359 - loss: 0.6582 - val_accuracy: 0.6667 - val_loss: 0.6477
Epoch 6/100
1/1 ━━━━━━━

In [11]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

2/2 - 0s - 15ms/step - accuracy: 0.7297 - loss: 0.6207


In [14]:
# !tensorboard --logdir="C:\Users\Windows 11\Desktop\Projetos\NN_tools\tensorflow\regression\logs\fit"